In [1]:
# done in Julia 0.5.0

In [2]:
output_dict = Dict(16=>(0), 1=>(1), 2=> (2), 3=>(3), 4=> (4), 5=> (5), 6 =>(6),
7=>(7), 8 =>(8), 9=>(9), 10 =>('A'), 11=>('B'), 12=>('C'), 13 =>('D'), 14 => ('E'), 15=> ('F'))

# This is used so that the final answer is easy for people to read.  
# Note that the actual numbering in Super Suduko is in Hex
# However a Sudoku puzzle has numerous empty cells in it and by convention an empty / sparse cell in a matrix is a zero
# Hence for coding purposes, we solve the puzzle using numbers {1, 2, 3, ..., 15, 16} 
# and then use the above mapping to convert the final answer back into the official numbering


Dict{Int64,Any} with 16 entries:
  2  => 2
  16 => 0
  11 => 'B'
  7  => 7
  9  => 9
  10 => 'A'
  8  => 8
  6  => 6
  4  => 4
  3  => 3
  5  => 5
  13 => 'D'
  14 => 'E'
  15 => 'F'
  12 => 'C'
  1  => 1

In [3]:
function diagonal_sum(x_tensor, k, i_max=16 )
    """
    Some Super Sudoku puzzles have an additional constraint along the diagonal of the matrix 
    (and on the 'fake diagonal' that goes from bottom left to top right of the matrix)
    
    This is the helper function for the constraint along the diagonal
    """
    running_sum = 0
    for i = 1:i_max
        running_sum += x_tensor[i,i,k]
    end
    return running_sum
end


function improper_diagonal_sum(x_tensor, k, i_max=16 )
    """
    This is the helper function for the constraint along the fake diagonal
    
    Recall that the constraint along the diagonal is optional, so this may or may not be in use
    """
    running_sum = 0
    for i = 1:i_max
        running_sum += x_tensor[i_max + 1 - i,i,k]
    end
    return running_sum
end


improper_diagonal_sum (generic function with 2 methods)

In [4]:
###################################################
USE_DIAGONAL_CONSTRAINT = false
# some Super Sudoku puzzles have the Diagonal Constraint active, while others do not
# set the above to = true, if you want it enabled

using JuMP
using Cbc

mymodel = Model(solver= CbcSolver())

A = readcsv("supporting_files/super_sudoku_sunday_oct2_.csv")

flatA = vec(A)

mini_size = 4
m = 16
n = 16 
k_max = 16
k_vec = collect(1:n)

assert(m==n)
assert(mini_size*mini_size - m == 0)
assert(size(A)[1] == m)
assert(size(A)[2] == n)

@variable(mymodel, 0<=y[i = 1:m*n]<=m)  

@variable(mymodel, x[i = 1:m*n*k_max], Bin) 
# @variable(mymodel, 0<= x[i = 1:m*n*k_max] <=1)
# note that the actual domain for the indicator variables in x are {0,1} i.e. binary
# however because the official problems typically have one and only one feasible solution we can frequently 
# get the exact solution by using [0,1] as the domain and solving this as an LP

y_box = reshape(y, (m, n)) * 1
x_tensor = reshape(x, (m, n, k_max))
# collect y values in a 16 x 16 matrix, and collect x values in 16 x 16 x 16 cube-like tensor
# the x values are really just indicator variable expansions for each of the 16 options associated with each cell in y

@constraint(mymodel, sum(x) == m*n) # i.e. one indicator per row per column 

for i = 1:n
    for j = 1:n
        @constraint(mymodel, y_box[i,j]  - dot(k_vec, vec(x_tensor[i,j,:])) == 0)
        # this constraint enforces our indicactor variables and the associated values showing up in y_box
        
        @constraint(mymodel, sum(x_tensor[i,j, :]) == 1)
        # note this is the core constraint that is "repeated" in multiple forms
        
        k_val = Int(round(A[i,j])) # this should clean up the floating point comparison issues associated with the below 
        if k_val > 0
            @constraint(mymodel, x_tensor[i,j, k_val] == 1)  
            ## This enforces agreement with pre-supplied / filled in cells, and our variables, where applicable
        end        
    end    
end


for i = 1:n
    for k = 1:k_max
        @constraint(mymodel, sum(x_tensor[i,:, k]) == 1)
        # note that this is the 'core constraint' that is repeated after we rotate the cube
    end
end


for j = 1:n
    for k = 1:k_max
        @constraint(mymodel, sum(x_tensor[:,j, k]) == 1)
        # note that this is the 'core constraint' that is repeated after we rotate the cube once more
    end
end

#####
# this last one is the trickiest...
# it is in place so that each submatrix / block has all possible numbers
# it is a touch awkward to do in 1-based indexing systems like Julia
for i = 0:(mini_size - 1)
    for j = 0:(mini_size - 1)
        for k = 1:k_max
            @constraint(mymodel, sum(x_tensor[mini_size*i + 1: mini_size*i + mini_size, mini_size*j + 1:mini_size*j + mini_size, k]) == 1)
        end
    end
end
#####

if USE_DIAGONAL_CONSTRAINT
    for k = 1:k_max
        @constraint(mymodel, diagonal_sum(x_tensor, k, m) == 1) 
        # standard diagonal from top left to bottom right of matrix
        @constraint(mymodel, improper_diagonal_sum(x_tensor, k, m) == 1)
        # improper diagonal from bottom left to top right of matrix        
    end
end

payoff_z = 0
# note that Sudoku is really a feasibility problem, so the payoff function can be whatever dummy function we like

@objective(mymodel, Min, payoff_z)
###########

# solve for the solution
solve(mymodel)

output = getvalue(y_box)

# the below is just some cleanup, to get the solution in an easy to use form
answer_matrix = round(Int64, output)

thing = convert(Array{Any,2}, answer_matrix)

m_rows, n_cols = size(thing)
for i = 1:m_rows
    for j = 1:n_cols
        newvalue_or_string = output_dict[thing[i,j]]
        thing[i,j] = newvalue_or_string
    end
end

using DataFrames
df = DataFrame(thing)
df

# writetable("Oct2output.dat", df, separator = ',', header = false)

# comment the above line out if you do not want to write it to a spreadsheet
# The data is not centered in the spreadsheet, but otherwise it looks ok to me
# just right click "output.data" and open in libre office calc and center the data

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16
1,A,B,5,C,0,6,2,E,1,8,9,F,4,7,3,D
2,E,F,D,2,3,B,5,C,A,0,4,7,9,1,6,8
3,6,1,4,8,F,A,9,7,C,D,B,3,E,0,2,5
4,7,3,9,0,8,D,4,1,5,E,6,2,B,A,C,F
5,3,5,0,E,2,8,A,B,9,C,F,6,1,4,D,7
6,D,2,8,6,7,1,F,9,4,A,3,5,0,C,E,B
7,9,7,1,F,4,C,E,6,0,B,2,D,3,5,8,A
8,C,A,B,4,5,0,D,3,E,7,1,8,F,6,9,2
9,4,8,F,B,C,2,6,A,3,9,5,1,D,E,7,0
10,5,0,6,D,E,F,B,4,7,2,8,A,C,3,1,9
